In [ ]:
# Import bibliotek
import os
import time
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Folder z obrazami
IMAGE_FOLDER = r"C:/Users/zonea/Desktop/WSB/seminarium/APP/test_tworzacy_excel_z_wynikami_modeli_bazowych"

# Ścieżka wyjściowa do pliku Excel
EXCEL_OUTPUT = r"C:/Users/zonea/Desktop/WSB/seminarium/APP/test_tworzacy_excel_z_wynikami_modeli_bazowych/wynik_testu.xlsx"

# Trzy serwisy do analizy:
SITES = [
    {
        "name": "AI Image Detector",
        "url": "https://aiimagedetector.org/",
        "file_input_selector": "input[type='file']",
        "result_selector": "h4.text-lg span.text-base",
        "analyze_button_selector": "//button[contains(text(),'Analyze')]",
        "invert": True 
    },
    {
        "name": "The Detector AI",
        "url": "https://thedetector.ai/",
        "file_input_selector": "input[type='file']",
        "result_selector": "p.text-gray-400",
        "analyze_button_selector": "//button[contains(text(),'Analyze Image')]",
        "invert": False
    },
    {
        "name": "AI Detect Content",
        "url": "https://aidetectcontent.com/ai-generated-image-detector/",
        "file_input_selector": "input[type='file']",
        "result_section_id": "result-section",
        "fake_percentage_id": "fake_percentage",
        "analyze_button_selector": "//button[contains(text(),'Check Image')]",
        "invert": False
    }
]

# Funkcje pomocznicze

def ensure_workbook(path):
    if not os.path.exists(path):
        df = pd.DataFrame(columns=[
            "nazwa_obrazu", "kategoria", "czy_ai",
            "ai_image_detector", "the_detector_ai", "ai_detect_content"
        ])
        df.to_excel(path, index=False)

def parse_filename(fn):
    parts = fn.lower().split('_')
    is_ai = 1 if parts[0] == "ai" else 0
    cat = parts[1] if len(parts) > 1 else "unknown"
    return cat, is_ai

def analyze_image_on_site(driver, site, img_path):
    driver.get(site["url"])
    wait = WebDriverWait(driver, 30)

    # upload
    inp = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, site["file_input_selector"])
    ))
    inp.send_keys(img_path)

    # click
    try:
        btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, site["analyze_button_selector"])
        ))
        btn.click()
    except:
        inp.send_keys(Keys.ENTER)

    # czekaj
    time.sleep(5)

    # różne sposoby odczytu wyników:
    if site["name"] == "AI Detect Content":
        # czekaj na sekcję z ID
        wait.until(EC.visibility_of_element_located(
            (By.ID, site["result_section_id"])
        ))
        txt = driver.find_element(By.ID, site["fake_percentage_id"]).text
    elif site["name"] == "The Detector AI":
        # p.text-gray-400, szukamy % w kilku próbach
        txt = ""
        for _ in range(3):
            elems = driver.find_elements(By.CSS_SELECTOR, site["result_selector"])
            txt = next((e.text for e in elems if "%" in e.text), "")
            if txt:
                break
            time.sleep(5)
        if not txt:
            txt = driver.execute_script("""
                return [...document.querySelectorAll('p.text-gray-400')]
                  .find(el=>el.innerText.includes('%'))?.innerText||""
            """)
    else:  # AI Image Detector
        elem = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, site["result_selector"])
        ))
        txt = elem.text.strip()

    # parsowanie liczby
    m = re.search(r'(\d+(?:\.\d+)?)', txt)
    if not m:
        return None
    val = float(m.group(1))
    if site.get("invert", False):
        val = 100 - val
    return val / 100.0

# -------------------------------
# GŁÓWNA PĘTLA
# -------------------------------

ensure_workbook(EXCEL_OUTPUT)
df_results = pd.read_excel(EXCEL_OUTPUT)

files = sorted(f for f in os.listdir(IMAGE_FOLDER)
               if f.lower().endswith((".jpg",".jpeg",".png")))

for fn in files:
    path = os.path.join(IMAGE_FOLDER, fn)
    cat, label = parse_filename(fn)
    print(f"\nAnaliza {fn} – kategoria={cat}, AI={label}")

    opts = webdriver.EdgeOptions()
    opts.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Edge(options=opts)

    # trzy wywołania:
    r0 = analyze_image_on_site(driver, SITES[0], path)
    r1 = analyze_image_on_site(driver, SITES[1], path)
    r2 = analyze_image_on_site(driver, SITES[2], path)

    driver.quit()

    new_row = pd.DataFrame([{
        "nazwa_obrazu": fn,
        "kategoria":    cat,
        "czy_ai":       label,
        "ai_image_detector": round(r0,3) if r0 is not None else None,
        "the_detector_ai":   round(r1,3) if r1 is not None else None,
        "ai_detect_content": round(r2,3) if r2 is not None else None
    }])

    df_results = pd.concat([df_results, new_row], ignore_index=True)

    df_results.to_excel(EXCEL_OUTPUT, index=False)
    print(f"  -> zapisano wynik dla {fn}")

print("\n Gotowe, wyniki zapisane w:", EXCEL_OUTPUT)